# Bot Python + Telegram

### Objetivo

Criar um script em Python que:
1.	Consulta o preço atual do BTC na Binance
2.	Envia o valor para o Telegram
3.	Protege os dados sensíveis usando .env
4.	Executa periodicamente com time.sleep()

### Pré-requisitos
1.	Criar um bot no Telegram via @BotFather
2.	Obter seu chat_id enviando uma mensagem ao bot e consultando a API
3.	Criar um arquivo .env com as credenciais
4.	Instalar bibliotecas necessárias

### Estrutura final do projeto

- /preco_btc_bot/
-  │
-  ├── bot_binance.py     ← seu script Python
-  └── .env               ← credenciais privadas

### Etapa 1 – Criar o Bot e pegar o Token
1.	Fale com @BotFather
2.	Envie /newbot, dê um nome e um username
3.	Anote o token do bot, que será algo como: 123456789:AAEJHJ23gakljhdgaskjhdgkashdgkj

### Etapa 2 – Descobrir o chat_id

Envie uma mensagem qualquer para seu bot no Telegram, depois execute este código:

In [ ]:
import requests

BOT_TOKEN = 'SEU:TOKEN'

# Retorna a última mensagem recebida pelo bot
response = requests.get(f'https://api.telegram.org/bot{BOT_TOKEN}/getUpdates')
print(response.json())

### Etapa 3 – Criar o arquivo .env

Crie um arquivo no mesmo diretório do seu script com o nome .env:

### Etapa 4 – Instalar dependências - se necessário

Use o pip: pip install requests python-dotenv

### Etapa 5 – Código Python

In [ ]:
import requests
import time
import os
from dotenv import load_dotenv

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()
BOT_TOKEN = os.getenv('BOT_TOKEN')
CHAT_ID = os.getenv('CHAT_ID')

# Obtém o preço do Bitcoin via API pública da Binance
def get_btc_price():
    url = 'https://api.binance.com/api/v3/ticker/price?symbol=BTCUSDT'
    response = requests.get(url)
    data = response.json()
    return float(data['price'])

# Envia uma mensagem para o Telegram com preço
def send_to_telegram(message, bot_token, chat_id):
    url = f'https://api.telegram.org/bot{bot_token}/sendMessage'
    payload = {
        'chat_id': chat_id,
        'text': message
    }
    response = requests.post(url, data=payload)
    return response.status_code

In [ ]:
# Loop infinito que envia o preço do Bitcoin a cada intervalo
while True:
    try:
        price = get_btc_price()
        message = f'O preço atual do Bitcoin (BTC/USDT) na Binance é: ${price:.2f}'
        status = send_to_telegram(message, BOT_TOKEN, CHAT_ID)
        print(f'Mensagem enviada com status {status} | {message}')
    except Exception as e:
        print(f"Erro ao enviar mensagem: {e}")

    # Aguarda 1 minuto antes de enviar novamente (60 segundos)
    time.sleep(60)

### Etapa 6 - Mandando os gráficos como imagem

In [ ]:
import os
import time
import requests
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from dotenv import load_dotenv

# --- Carrega variáveis de ambiente ---
load_dotenv()
BOT_TOKEN = os.getenv("BOT_TOKEN")
CHAT_ID = os.getenv("CHAT_ID")

# --- Função para obter dados históricos do par especificado ---
def get_klines(symbol="BTCUSDT", interval="1h", limit=100):
    url = "https://api.binance.com/api/v3/klines"
    params = {"symbol": symbol, "interval": interval, "limit": limit}
    response = requests.get(url, params=params)
    data = response.json()
    df = pd.DataFrame(data, columns=[
        "timestamp", "open", "high", "low", "close", "volume",
        "_", "_", "_", "_", "_", "_"
    ])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df.set_index("timestamp", inplace=True)
    df["close"] = df["close"].astype(float)
    return df[["close"]]

# --- Função para gerar o gráfico e salvar como imagem PNG ---
def generate_plot():
    btc = get_klines("BTCUSDT")
    eth = get_klines("ETHUSDT")

    eth_btc = eth["close"] / btc["close"]
    eth_btc.name = "ETH/BTC"

    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        subplot_titles=("BTC/USDT", "ETH/USDT", "ETH/BTC"),
        vertical_spacing=0.05
    )

    fig.add_trace(go.Scatter(x=btc.index, y=btc["close"], name="BTC/USDT", line=dict(color="orange")), row=1, col=1)
    fig.add_trace(go.Scatter(x=eth.index, y=eth["close"], name="ETH/USDT", line=dict(color="blue")), row=2, col=1)
    fig.add_trace(go.Scatter(x=eth_btc.index, y=eth_btc, name="ETH/BTC", line=dict(color="green")), row=3, col=1)

    fig.update_layout(
        height=900,
        width=900,
        title="Mercado Cripto: BTC, ETH e ETH/BTC - https://om-qs.com",
        showlegend=False,
        margin=dict(t=60, b=20)
    )

    fig.write_image("btc_eth_chart.png")

# --- Envia imagem via Telegram ---
def send_image_to_telegram(bot_token, chat_id, image_path, caption="Atualização de mercado cripto"):
    url = f"https://api.telegram.org/bot{bot_token}/sendPhoto"
    with open(image_path, "rb") as photo:
        files = {"photo": photo}
        data = {"chat_id": chat_id, "caption": caption}
        response = requests.post(url, files=files, data=data)
    return response.status_code

In [ ]:
# --- LOOP de envio periódico ---
while True:
    try:
        generate_plot()
        status = send_image_to_telegram(BOT_TOKEN, CHAT_ID, "btc_eth_chart.png")
        print(f"Imagem enviada com status {status}")
    except Exception as e:
        print(f"Erro: {e}")
        
    # Espera 1 minuto antes de repetir (modifique conforme necessário)
    time.sleep(60)

Se quiser testar manualmente sem loop, execute somente:

In [ ]:
generate_plot()
send_image_to_telegram(BOT_TOKEN, CHAT_ID, "btc_eth_chart.png")